# Sparks

## Introduction
[Sparks](https://github.com/LIVENlab/Sparks) is an open-source repository designed to facilitate the integration of life cycle assessment (LCA) data with energy system configurations generated by [Calliope](https://calliope.readthedocs.io/en/stable/#). It creates an input file for [ENBIOS](https://pypi.org/project/enbios/), adapting units, structuring data into a dendrogram tree-like format, and optionally aggregating regions and subregions.

## Import Sparks


In [22]:
from Sparks.util.base import SoftLink

In order to run Sparks, you have to define a `Basefile`, an excel file that maps LCI data, energy conversions, and hierarchical relationships in order to create an input for ENBIOS.


In [23]:
import pandas as pd
df = pd.read_excel(r'C:\Users\altz7\PycharmProjects\Sparks_v2\testing\data_test\basefile.xlsx', sheet_name='Processors')
df.head()


,Processor,Region,@SimulationCarrier,ParentProcessor,@SimulationToEcoinventFactor,Ecoinvent_key_code,File_source
0,wind_onshore,PRT,electricity,Electricity_Generation_maintenance,1.000000e+09,81174ec2c20931c1a36f65c654bbd11e,nameplate_capacity.csv
1,wind_onshore,PRT,electricity,Electricity_Generation_Operation,1.000000e+09,81174ec2c20931c1a36f65c654bbd11e,flow_out_sum.csv
2,wind_offshore,PRT,electricity,Electricity_Generation_maintenance,1.000000e+09,81174ec2c20931c1a36f65c654bbd11e,nameplate_capacity.csv
3,hydro_reservoir,PRT,electricity,Electricity_Generation_Operation,1.000000e+09,c868c4688fbf78f5ca3787ac3d83312b,flow_out_sum.csv
4,ccgt,PRT,electricity,Electricity_Generation_Operation,1.000000e+09,4833b6352dfe15c95ae46fd280371cd3,flow_out_sum.csv


- `Processor` --> is the name of the technology in the energy system model output
- `Region` --> Region in the energy data 
- `@SimulationCarrier` --> Carrier in the energy data
- `ParentProcessor` --> Hierarchical parent. 
- `@SimulationToEcoinventFactor` --> Conversion factor mapping the relationship between the units of the energy data and the inventory
- `@Ecoinvent_key_code` --> BW25 database code. If multiples databases have the same code, it will rise a warning and select the first item.
- `File_source` --> File where the input data will be extracted from. It must be present in the data folder


### Initialize Sparks
Pass a `path` to the folder with the `basefile` and the different energy system's outputs', along with a BW25 project

In [24]:

enbios_mod = SoftLink(r'C:\Users\altz7\PycharmProjects\Sparks_v2\testing\data_test',
                    'Hydrogen_SEEDS')

Project and Database existing...


Let's transform the data, mapping first the LCI data with the energy system's data 

In [25]:
enbios_mod.preprocess()

C:\Users\altz7\PycharmProjects\Sparks_v2\Sparks\generic\generic_dataclass.py:49: Warning: More than one activity found with code CHP_hydrogen_2050
  warnings.warn(f"More than one activity found with code {key}",Warning)
C:\Users\altz7\PycharmProjects\Sparks_v2\Sparks\generic\generic_dataclass.py:58: Warning: 
CHP_hydrogen_2050_HEAT_1 not found in the database. Please check your database / basefile.
This activity won't be included.
  warnings.warn(message, Warning)


function preprocess executed in 19.48020911216736 seconds


C:\Users\altz7\PycharmProjects\Sparks_v2\Sparks\generic\generic_dataclass.py:58: Warning: 
06412182 not found in the database. Please check your database / basefile.
This activity won't be included.
  warnings.warn(message, Warning)


[Sparks](https://github.com/LIVENlab/Sparks) will notify missing activities in the database as well as activities present in more than one database. To see what's happening in this face, you can run the following code:

In [26]:
enbios_mod.preprocessed_units

,scenarios,locs,techs,full_name,carriers,new_units,energy_value,aliases
0,0,PRT_1,battery,battery_electricity__flow_out_sum___PRT_1,electricity,kilogram,2.173270e+04,battery__electricity___PRT_1
1,0,PRT_2,battery,battery_electricity__flow_out_sum___PRT_2,electricity,kilogram,2.240146e+04,battery__electricity___PRT_2
2,0,PRT_1,biofuel_boiler,biofuel_boiler_heat__flow_out_sum___PRT_1,heat,megajoule,7.330219e+06,biofuel_boiler__heat___PRT_1
3,0,PRT_2,biofuel_boiler,biofuel_boiler_heat__flow_out_sum___PRT_2,heat,megajoule,1.113732e+06,biofuel_boiler__heat___PRT_2
4,0,PRT_1,biofuel_to_diesel,biofuel_to_diesel_diesel__flow_out_sum___PRT_1,diesel,kilogram,1.187307e+08,biofuel_to_diesel__diesel___PRT_1
...,...,...,...,...,...,...,...,...
34814,268,PRT-9_1,wind_offshore,wind_offshore_electricity__nameplate_capacity_...,electricity,kilowatt hour,2.139181e+07,wind_offshore__electricity___PRT-9_1
34815,269,PRT-11_1,wind_offshore,wind_offshore_electricity__nameplate_capacity_...,electricity,kilowatt hour,1.925102e+08,wind_offshore__electricity___PRT-11_1
34816,269,PRT-9_1,wind_offshore,wind_offshore_electricity__nameplate_capacity_...,electricity,kilowatt hour,9.903914e+09,wind_offshore__electricity___PRT-9_1
34817,270,PRT-11_1,wind_offshore,wind_offshore_electricity__nameplate_capacity_...,electricity,kilowatt hour,3.885973e+09,wind_offshore__electricity___PRT-11_1


Sparks groups and classifies activities, creates individual names to avoid errors in enbios, and transform the units according to the inventory data.

## Create an input for ENBIOS

According to the methods, and hierarchy defined in the basefile, we can create an input for enbios like this:

In [27]:
enbios_mod.data_for_ENBIOS(smaller_vers=False, path_save=r'test.json')

C:\Users\altz7\PycharmProjects\Sparks_v2\Sparks\generic\generic_dataclass.py:120: UserWarning: leafs not found for Last Tree Branch Thermal_storage, at n-2. This  can induce critical errors when using this data in enbios. Please, check the dendrogram structure
  warnings.warn(f"leafs not found for Last Tree Branch {self.name}, at {self.level}. This  can induce critical errors when using this data in enbios. Please, check the dendrogram structure")


Input data for ENBIOS created


Sparks will notify possible problems on the definition of the hierarchy that could lead to critical errors in ENBIOS. 

If you want to check the structure of this file, you can use the following [JSON schema](https://github.com/LIVENlab/enbios/blob/main/data/schema/experiment.schema.gen.json) and a [validator](https://www.jsonschemavalidator.net/)

In [2]:
import json

with open('test.json', 'r', encoding="utf-8") as outfile:
    data =json.load(outfile)
data.keys()
data['hierarchy']

{'name': 'Energysystem',
 'aggregator': 'sum',
 'children': [{'name': 'Generation',
   'aggregator': 'sum',
   'children': [{'name': 'Electricity_generation',
     'aggregator': 'sum',
     'children': [{'name': 'Electricity_Generation_Operation',
       'aggregator': 'sum',
       'children': [{'name': 'wind_onshore_electricity__flow_out_sum___PRT-10_1',
         'adapter': 'bw',
         'config': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
        {'name': 'wind_onshore_electricity__flow_out_sum___PRT-11_1',
         'adapter': 'bw',
         'config': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
        {'name': 'wind_onshore_electricity__flow_out_sum___PRT-12_1',
         'adapter': 'bw',
         'config': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
        {'name': 'wind_onshore_electricity__flow_out_sum___PRT-14_1',
         'adapter': 'bw',
         'config': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
        {'name': 'wind_onshore_electricity__flow_out_sum___PRT-1